- add number of users per date

In [1]:
!pip install jpholiday 

In [2]:
import os
import warnings
import jpholiday
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
from datetime import timedelta
from sklearn import preprocessing
from lightgbm import LGBMClassifier
from tqdm.notebook import tqdm_notebook
from sklearn.metrics import average_precision_score
from sklearn.model_selection import StratifiedKFold
warnings.filterwarnings('ignore')
tqdm_notebook.pandas(desc="progress: ")

In [3]:
DIR = "../input/20201115at/"
train = pd.read_csv(DIR+"train.csv")
test = pd.read_csv(DIR+"test.csv")
campaign = pd.read_csv(DIR+"campaign.csv")

#advertiser_video = pd.read_csv(DIR+"advertiser_video.csv")
#map_game_feed_native_video_assets = pd.read_csv(DIR+'map_game_feed_native_video_assets.csv')
#advertiser_converted_video = pd.read_csv(DIR+"advertiser_converted_video.csv")

In [4]:
#cols = ["mst_advertiser_video_id", "mst_game_feed_id", "mst_video_template_id"]
#advertiser_converted_video = advertiser_converted_video[~advertiser_converted_video[cols].duplicated(keep="last")]

# preprocess

In [5]:
# with campaign
train = train.merge(campaign, left_on="campaign_id", right_on = "id", how="left").drop(["mst_advertiser_id", "id"], axis=1)
test = test.merge(campaign, left_on="campaign_id", right_on = "id", how="left").drop(["mst_advertiser_id", "id"], axis=1)

In [6]:
def merge_all(
    df: pd.DataFrame, map_game_feed_native_video_assets: pd.DataFrame,
    advertiser_video: pd.DataFrame, advertiser_converted_video: pd.DataFrame,
) -> pd.DataFrame:
    # merge res and map_game_feed_native_video_assets
    res = pd.merge(df,map_game_feed_native_video_assets,left_on="game_feed_id",right_on="mst_game_feed_id",how="left",
    ).drop(columns=["mst_game_feed_id"])  
    # remove map_game_feed_native_video_assets keys

    # merge res and advertiser_video (horizontal case)
    horizontal = advertiser_video.copy()
    left_keys = ["horizontal_mst_advertiser_video_id", "advertiser_id"]
    right_keys = ["id", "mst_advertiser_id"]
    horizontal.columns = [f"horizontal_{c}" if c not in right_keys else c for c in horizontal.columns]
    res = pd.merge(res, horizontal, left_on=left_keys, right_on=right_keys, how="left").drop(columns=right_keys) 
    # remove advertiser_video keys
    
    # merge res and advertiser_video (vertical case)
    vertical = advertiser_video.copy()
    left_keys = ["vertical_mst_advertiser_video_id", "advertiser_id"]
    right_keys = ["id", "mst_advertiser_id"]
    vertical.columns = [f"vertical_{c}" if c not in right_keys else c for c in vertical.columns]
    res = pd.merge(res, vertical, left_on=left_keys, right_on=right_keys, how="left").drop(columns=right_keys) 
    # remove advertiser_video keys

    # merge res and advertiser_converted_video (horizontal case)
    left_keys = ["horizontal_mst_advertiser_video_id", "game_feed_id", "video_template_id",]
    right_keys = ["mst_advertiser_video_id","mst_game_feed_id", "mst_video_template_id",]
    horizontal = advertiser_converted_video.copy()
    horizontal.columns = [ f"horizontal_converted_{c}" if c not in right_keys else c for c in horizontal.columns]
    res = pd.merge(res, horizontal, left_on=left_keys, right_on=right_keys, how="left").drop(columns=right_keys)
    # remove advertiser_converted_video keys
    
    # merge res and advertiser_converted_video (vertical case)
    left_keys = ["vertical_mst_advertiser_video_id","game_feed_id","video_template_id",]
    right_keys = ["mst_advertiser_video_id","mst_game_feed_id","mst_video_template_id",]
    vertical = advertiser_converted_video.copy()
    vertical.columns = [f"vertical_converted_{c}" if c not in right_keys else c for c in vertical.columns]
    res = pd.merge(res, vertical, left_on=left_keys, right_on=right_keys, how="left").drop(columns=right_keys)
    # remove advertiser_converted_video keys

    return res

#train = merge_all(train, map_game_feed_native_video_assets, advertiser_video, advertiser_converted_video)
#test = merge_all(test, map_game_feed_native_video_assets, advertiser_video, advertiser_converted_video)

#train["duration"] = train["vertical_duration"].fillna(0) + train["horizontal_duration"].fillna(0)
#train.drop(["vertical_duration", "horizontal_duration"], axis=1, inplace=True)

#test["duration"] = test["vertical_duration"].fillna(0) + test["horizontal_duration"].fillna(0)
#test.drop(["vertical_duration", "horizontal_duration"], axis=1, inplace=True)

# filesize
#train["filesize"] = train["vertical_file_size"].fillna(0) + train["horizontal_file_size"].fillna(0)
#train.drop(["vertical_file_size", "horizontal_file_size"], axis=1, inplace=True)

#test["filesize"] = test["vertical_file_size"].fillna(0) + test["horizontal_file_size"].fillna(0)
#test.drop(["vertical_file_size", "horizontal_file_size"], axis=1, inplace=True)

# converted filesize
#train["converted_filesize"] = train["vertical_converted_file_size"].fillna(0.0) + train["horizontal_converted_file_size"].fillna(0)
#train.drop(["vertical_converted_file_size", "horizontal_converted_file_size"], axis=1, inplace=True)

#test["converted_filesize"] = test["horizontal_converted_file_size"].fillna(0.0) + test["vertical_converted_file_size"].fillna(0)
#test.drop(["vertical_converted_file_size", "horizontal_converted_file_size"], axis=1, inplace=True)

# bitrate
#train["bitrate"] = train["vertical_converted_bitrate"].fillna(0) + train["horizontal_converted_bitrate"].fillna(0)
#train.drop(["vertical_converted_bitrate", "horizontal_converted_bitrate"], axis=1, inplace=True)

#test["bitrate"] = test["vertical_converted_bitrate"].fillna(0) + test["horizontal_converted_bitrate"].fillna(0)
#test.drop(["vertical_converted_bitrate", "horizontal_converted_bitrate"], axis=1, inplace=True)

In [7]:
train["imp_at"] = pd.to_datetime(train["imp_at"])
test["imp_at"] = pd.to_datetime(test["imp_at"])
train["date"] = train["imp_at"].dt.date
test["date"] = test["imp_at"].dt.date

In [8]:
train["click_or_view_at"] = pd.to_datetime(train["click_or_view_at"])
train["cv_at"] = pd.to_datetime(train["cv_at"])

In [9]:
# drop because distribution is different
train.drop(['os_version'], axis=1, inplace=True)
test.drop(['os_version'], axis=1, inplace=True)

# noise features?
train.drop(['user_type_id', "category_id",  "video_template_id", "country_code", "game_feed_asset_type_id"], axis=1, inplace=True)
test.drop(['user_type_id', "category_id",  "video_template_id", "country_code", "game_feed_asset_type_id"], axis=1, inplace=True)

In [10]:
object_feats = [i for i in train.columns if "object" == str(train[i].dtype) 
                and i != "request_id" and i != "uid" and i != "date"]

def get_non_overlapping(train: pd.DataFrame, test: pd.DataFrame, column: str):
    only_in_train = set(train[column].unique()) - set(test[column].unique())
    only_in_test = set(test[column].unique()) - set(train[column].unique())
    non_overlapping = only_in_train.union(only_in_test)
    return non_overlapping

def category2num(train: pd.DataFrame, test: pd.DataFrame, columns: list):
    train_ = train.copy()
    test_ = test.copy()
    for column in columns:
        non_overlapping = get_non_overlapping(train, test, column)
        try:
            if train[column].dtype == np.dtype("O"):
                # dtypeがobjectなら欠損は'missing' クラスにする
                train_[column] = train[column].fillna("missing")
                test_[column] = test[column].fillna("missing")
                train_[column] = train_[column].map(lambda x: x if x not in non_overlapping else "other")
                test_[column] = test_[column].map(lambda x: x if x not in non_overlapping else "other")
            else:
                # dtypeがint/floatなら欠損は'-1'とする
                train_[column] = train[column].fillna(-1)
                test_[column] = test[column].fillna(-1)
                train_[column] = train_[column].map(lambda x: x if x not in non_overlapping else -2)
                test_[column] = test_[column].map(lambda x: x if x not in non_overlapping else -2)

            le = preprocessing.LabelEncoder()
            concatenated = pd.concat([train_, test_], axis=0).reset_index(drop=True)
            le.fit(concatenated[column])
            train_[column] = le.transform(train_[column])
            test_[column] = le.transform(test_[column])
        except Exception:
            import pdb
            pdb.set_trace()
    return train_, test_

train, test = category2num(train, test, object_feats)

In [11]:
#train["click_or_view_day"] = train["click_or_view_at"].dt.dayofweek
#click_view_df = train.groupby("uid")["click_or_view_day"].value_counts().unstack(fill_value=0)
#click_view_df.columns = ["click_or_view"+str(i) for i in range(7)]
#click_view_df = click_view_df.reset_index(drop=False)
#train.drop("click_or_view_day", axis=1, inplace=True)

#train = pd.merge(train, click_view_df, on="uid", how="left")
#test = pd.merge(test, click_view_df, on="uid", how="left")

#train["cv_hour"] = train["cv_at"].dt.hour
#cv_df = train.groupby("uid")["cv_hour"].value_counts().unstack(fill_value=0)
#cv_df.columns = ["cv_hour"+str(i) for i in range(24)]
#cv_df = cv_df.reset_index(drop=False)
#train.drop("cv_hour", axis=1, inplace=True)

#train = pd.merge(train, click_view_df, on="uid", how="left")
#test = pd.merge(test, click_view_df, on="uid", how="left")

#train = pd.merge(train, cv_df, on="uid", how="left")
#test = pd.merge(test, cv_df, on="uid", how="left")

In [12]:
# drop because the exist only in train
train.drop(['click_or_view_at', 'cv_at','last_cv_interval', 'last_paid_interval'], axis=1, inplace=True)

# feature engineering

In [13]:
train["type"] = "train"
test["type"] = "test"
all_df = pd.concat([train, test])
all_df = all_df.sort_values(["uid", "imp_at"]).reset_index(drop=True)

In [14]:
all_df["count"] = 1
all_df["count"] = all_df.groupby("uid")["count"].cumsum()

In [15]:
def _create_whole_imp_at_feature(whole_df, var) -> pd.DataFrame:

    imp_at = pd.to_datetime(whole_df['imp_at']) + timedelta(hours=9) # utc -> asia/tokyo
    out_df = pd.DataFrame()
    out_df['hours'] = imp_at.dt.hour
    out_df['dayofweek'] = imp_at.dt.dayofweek
    out_df['hour_zone'] = pd.cut(out_df['hours'].values, bins=[-np.inf, 6, 12, 18, np.inf]).codes

    def _create_pivot(input_df, c, column='dayofweek', values='hours'):
        _df = pd.pivot_table(data=input_df, index=[c], columns=[column], values=values, aggfunc='count')
        _df = _df.fillna(0)
        _df.columns = [column + '=' + str(x) for x in _df.columns]

        # index ごとに正規化して割合にする
        _df = _df.div(_df.sum(axis=1), axis=0)
        return pd.merge(input_df[c], _df, on=c, how='left').drop(columns=[c]).add_prefix(c + '_')

    # uid での集約
    for c in [var]:
        _df = out_df.copy()
        _df[c] = whole_df[c]
        _feat = pd.concat([
            _create_pivot(_df, c),
            _create_pivot(_df, c, column='hour_zone', values='dayofweek')  
        ], axis=1)
        out_df = pd.concat([out_df, _feat], axis=1)

    out_df['request_id'] = whole_df['request_id']
    return out_df

user_imp_df = _create_whole_imp_at_feature(all_df, "uid")
user_imp_df.drop(["hours","dayofweek","hour_zone"], axis=1, inplace=True)
all_df = all_df.merge(user_imp_df, on="request_id", how="left")

In [16]:
camp_num = all_df.groupby("date")["campaign_id"].nunique().reset_index(drop=False)
camp_num.columns=["date", "camp_num"]
all_df = all_df.merge(camp_num, on="date", how="left")

user_num = all_df.groupby("date")["uid"].nunique().reset_index(drop=False)
user_num.columns=["date", "user_num"]
all_df = all_df.merge(user_num, on="date", how="left")

all_df.drop("date", axis=1, inplace=True)

In [17]:
#all_df["date"] = all_df["imp_at"].dt.date
#all_df["dif_login"] = all_df.groupby("uid")["date"].progress_apply(lambda x: x.diff(1))
#all_df["dif_login"] = all_df["dif_login"].progress_apply(lambda x: x.days)
#all_df["dif_login"] = all_df["dif_login"].fillna(0)
#all_df["total_logdays"] = all_df["dif_login"].progress_apply(lambda x: x if x ==0 or x==1 else 1)
#all_df["total_logdays"] = all_df.groupby("uid")["total_logdays"].progress_apply(lambda x: x.cumsum()) + 1
#all_df.drop(["date", "dif_login"], axis=1, inplace=True)

In [18]:
train = all_df[all_df.type=="train"].drop("type", axis=1).reset_index(drop=True)
test = all_df[all_df.type=="test"].drop("type", axis=1).reset_index(drop=True)

In [19]:
train = train.sort_values("imp_at").reset_index(drop=True)
test = test.sort_values("imp_at").reset_index(drop=True)

In [20]:
ce_feats = ["uid"]
for c in ce_feats:
    freq = pd.concat([train[c], test[c]]).value_counts()
    train[c] = train[c].map(freq)
    test[c] = test[c].map(freq)

# separation for validation

In [21]:
train.drop(['request_id'], axis=1, inplace=True)
test.drop(['request_id'], axis=1, inplace=True)

In [22]:
mask = train.imp_at > "2020-05-21"
valid_df = train.loc[mask].copy()
train_df = train.loc[~mask].copy()

In [23]:
train = train.drop(["imp_at"], axis=1) 
test = test.drop(["imp_at"], axis=1) 

valid_df = valid_df.drop(["imp_at"], axis=1)
train_df = train_df.drop(["imp_at"], axis=1)

In [24]:
test = test.drop("target", axis=1)

In [25]:
train.shape, test.shape 

((1997595, 39), (390095, 38))

In [26]:
train.columns

Index(['target', 'adnw_id', 'adspot_id', 'adspot_video_format_id',
       'advertiser_id', 'app_id', 'auction_type_id', 'campaign_id',
       'first_login_interval', 'frequency', 'from_click', 'game_feed_id',
       'game_template_id', 'header_bidding', 'is_interstitial', 'item_id',
       'last_login_interval', 'login_frequency', 'max_login_interval',
       'media_app_id', 'os', 'pos', 'uid', 'mst_advertiser_order_id',
       'mst_user_type_id', 'count', 'uid_dayofweek=0', 'uid_dayofweek=1',
       'uid_dayofweek=2', 'uid_dayofweek=3', 'uid_dayofweek=4',
       'uid_dayofweek=5', 'uid_dayofweek=6', 'uid_hour_zone=0',
       'uid_hour_zone=1', 'uid_hour_zone=2', 'uid_hour_zone=3', 'camp_num',
       'user_num'],
      dtype='object')

In [27]:
object_feats = [i for i in object_feats if i in train.columns] 
object_feats

['advertiser_id',
 'app_id',
 'campaign_id',
 'game_feed_id',
 'game_template_id',
 'media_app_id',
 'mst_advertiser_order_id']

In [28]:
del all_df, user_imp_df

# stratified lgb

In [29]:
# stratified k fold
n_folds = 5
def pr_auc_metric(y_predicted, y_true):
    return 'pr_auc', average_precision_score(y_true.get_label(), y_predicted), True

def modelling(seed):
    print("seed:", seed)
    lgbm_params = {'objective': 'binary', 'metric': 'None', 'boosting_type': 'gbdt', 
               'tree_learner': 'serial', 'learning_rate': 0.1, "num_leaves": 10, 'random_seed':seed,'max_depth': 5}
    
    y = train.target.copy()
    X = train.drop(['target'],axis=1).copy()
    
    skf=StratifiedKFold(n_splits = n_folds, shuffle=True, random_state=0)
    
    valid = np.zeros([X.shape[0]])
    features_list = [i for i in X.columns]
    pred_value = np.zeros(test.shape[0])
    feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
    
    for i , (train_index, val_index) in enumerate(skf.split(X, y)):
        print("fold: ", i)
        
        X_train = X.iloc[train_index,:]
        y_train = y.iloc[train_index]

        X_valid = X.iloc[val_index,:]
        y_valid = y.iloc[val_index]
        
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
        
        model = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
           num_boost_round=10000,early_stopping_rounds=20,verbose_eval = 30,
                          feval=pr_auc_metric, categorical_feature = sorted(object_feats))
        
        valid_predict = model.predict(X_valid, num_iteration = model.best_iteration)
        valid[val_index] = valid_predict
        pred_value += model.predict(test, num_iteration = model.best_iteration)
        feature_importance_df["Fold_"+str(i+1)] = model.feature_importance()
    
    feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]
    
    return pred_value, valid, feature_importance_df

validation = np.zeros([train.shape[0]])
pred_value = np.zeros([test.shape[0]])
seeds = [0]
for seed_ in seeds:
    test_pred, oof_predict, feature_importance_df = modelling(seed_)
    validation += oof_predict / len(seeds)
    pred_value += test_pred / len(seeds)

score = average_precision_score(train.target, validation)
print(score)

seed: 0
fold:  0
Training until validation scores don't improve for 20 rounds
[30]	training's pr_auc: 0.268576	valid_1's pr_auc: 0.263582
[60]	training's pr_auc: 0.290141	valid_1's pr_auc: 0.277924
[90]	training's pr_auc: 0.304185	valid_1's pr_auc: 0.284539
[120]	training's pr_auc: 0.31375	valid_1's pr_auc: 0.288305
[150]	training's pr_auc: 0.321897	valid_1's pr_auc: 0.290957
[180]	training's pr_auc: 0.32895	valid_1's pr_auc: 0.29254
[210]	training's pr_auc: 0.335481	valid_1's pr_auc: 0.294169
[240]	training's pr_auc: 0.342901	valid_1's pr_auc: 0.29563
[270]	training's pr_auc: 0.348775	valid_1's pr_auc: 0.296857
[300]	training's pr_auc: 0.354791	valid_1's pr_auc: 0.297432
[330]	training's pr_auc: 0.360768	valid_1's pr_auc: 0.298033
Early stopping, best iteration is:
[334]	training's pr_auc: 0.361518	valid_1's pr_auc: 0.298157
fold:  1
Training until validation scores don't improve for 20 rounds
[30]	training's pr_auc: 0.266294	valid_1's pr_auc: 0.265154
[60]	training's pr_auc: 0.29022	

In [30]:
feature_importance_df.sort_values("Average", ascending=False)

,Feature,Fold_1,Fold_2,Fold_3,Fold_4,Fold_5,Average,Std,Cv
4,app_id,783,490,620,614,577,616.8,95.186974,0.154324
6,campaign_id,441,352,414,398,383,397.6,29.816774,0.074992
18,media_app_id,389,264,313,289,282,307.4,43.720018,0.142225
10,game_feed_id,234,146,180,172,149,176.2,31.701104,0.179915
3,advertiser_id,199,145,153,151,160,161.6,19.303886,0.119455
21,uid,166,143,154,159,159,156.2,7.626270,0.048824
22,mst_advertiser_order_id,110,86,85,76,80,87.4,11.859174,0.135688
24,count,99,75,82,75,83,82.8,8.772685,0.105950
15,last_login_interval,78,49,58,50,50,57.0,10.990905,0.192823
7,first_login_interval,61,34,49,52,49,49.0,8.694826,0.177445


# ts lgb

In [31]:
def pr_auc_metric(y_predicted, y_true):
    return 'pr_auc', average_precision_score(y_true.get_label(), y_predicted), True

def modelling_ts(seed):
    print("seed:", seed)
    lgbm_params = {'objective': 'binary', 'metric': 'None', 'boosting_type': 'gbdt', 
               'tree_learner': 'serial', 'learning_rate': 0.1, "num_leaves": 10, 'random_seed':seed,'max_depth': 5}
    y_train = train_df.target.copy()
    X_train = train_df.drop(['target'],axis=1).copy()
    y_valid = valid_df.target.copy()
    X_valid = valid_df.drop(['target'],axis=1).copy()
    
    valid_lgb = pd.DataFrame(np.zeros([X_train.shape[0]]))
    real = np.array([])
    features_list = [i for i in X_train.columns]
    feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
        
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
        
    model = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
       num_boost_round=10000,early_stopping_rounds=20,verbose_eval = 10, feval=pr_auc_metric, categorical_feature = sorted(object_feats))
    
    valid_predict = model.predict(X_valid, num_iteration = model.best_iteration)
    feature_importance_df["Importance"] = model.feature_importance()
        
    score = average_precision_score(y_valid, valid_predict)
    print("average precision score = {}".format(score))
    pred_value = model.predict(test, num_iteration = model.best_iteration)
    
    return pred_value, valid_predict, feature_importance_df

validation_ts = np.zeros([valid_df.shape[0]])
pred_value_ts = np.zeros([test.shape[0]])
seeds = [0]
for seed_ in seeds:
    test_pred, oof_predict, feature_importance_df = modelling_ts(seed_)
    validation_ts += oof_predict / len(seeds)
    pred_value_ts += test_pred / len(seeds)

ts_score = average_precision_score(valid_df.target, validation_ts)
print(ts_score)

seed: 0
Training until validation scores don't improve for 20 rounds
[10]	training's pr_auc: 0.253792	valid_1's pr_auc: 0.194218
[20]	training's pr_auc: 0.267442	valid_1's pr_auc: 0.201826
[30]	training's pr_auc: 0.279788	valid_1's pr_auc: 0.208617
[40]	training's pr_auc: 0.289771	valid_1's pr_auc: 0.21245
[50]	training's pr_auc: 0.298863	valid_1's pr_auc: 0.216081
[60]	training's pr_auc: 0.304332	valid_1's pr_auc: 0.217966
[70]	training's pr_auc: 0.308655	valid_1's pr_auc: 0.219414
[80]	training's pr_auc: 0.313227	valid_1's pr_auc: 0.22076
[90]	training's pr_auc: 0.316846	valid_1's pr_auc: 0.221405
[100]	training's pr_auc: 0.321304	valid_1's pr_auc: 0.222297
[110]	training's pr_auc: 0.325226	valid_1's pr_auc: 0.223388
[120]	training's pr_auc: 0.329264	valid_1's pr_auc: 0.224419
[130]	training's pr_auc: 0.331815	valid_1's pr_auc: 0.225099
[140]	training's pr_auc: 0.33514	valid_1's pr_auc: 0.225492
[150]	training's pr_auc: 0.338325	valid_1's pr_auc: 0.225752
[160]	training's pr_auc: 0.3

In [32]:
feature_importance_df.sort_values("Importance", ascending=False)

,Feature,Importance
4,app_id,375
6,campaign_id,299
18,media_app_id,166
21,uid,129
3,advertiser_id,96
10,game_feed_id,87
22,mst_advertiser_order_id,74
24,count,57
23,mst_user_type_id,39
15,last_login_interval,36


# stratified xgboost

In [33]:
# https://xgboost.readthedocs.io/en/latest/parameter.html
def pr_auc_metric(y_predicted, y_true):
    return 'pr_auc', - average_precision_score(y_true.get_label(), y_predicted)

n_folds=5
def modelling_xgb(seed):
    y = train.target
    X = train.drop(['target'],axis=1)
    xgb_params = {"objective" : "binary:logistic", "max_depth" : 5, "learning_rate" : 0.1,"tree_method": "gpu_hist"}
    
    skf=StratifiedKFold(n_splits = n_folds, shuffle=True, random_state=0)
    
    valid = np.zeros([X.shape[0]])
    features_list = [i for i in X.columns]
    pred_value = np.zeros(test.shape[0])
    X_test = xgb.DMatrix(test.copy())
    
    for i , (train_index, val_index) in enumerate(skf.split(X, y)):
        print("fold: ", i)
        
        X_train = X.iloc[train_index,:]
        y_train = y.iloc[train_index]

        X_valid = X.iloc[val_index,:]
        y_valid = y.iloc[val_index]
        
        xgb_train = xgb.DMatrix(X_train, label = y_train)
        xgb_eval = xgb.DMatrix(X_valid, label = y_valid)
        watchlist = [(xgb_train, "train"), (xgb_eval, "eval")]
        num_boost_round = 100000
        
        model = xgb.train(
        xgb_params, xgb_train, num_boost_round, watchlist,
        early_stopping_rounds=10, verbose_eval = 50,
        feval=pr_auc_metric,
    )
                
        valid_predict = model.predict(xgb_eval, ntree_limit = model.best_ntree_limit)
        valid[val_index] = valid_predict
        pred_value += model.predict(X_test, ntree_limit = model.best_ntree_limit)
    
    return pred_value, valid

validation_xgb = np.zeros([train.shape[0]])
pred_value_xgb = np.zeros([test.shape[0]])
seeds = [0]
for seed_ in seeds:
    test_pred, oof_predict = modelling_xgb(seed_)
    validation_xgb += oof_predict / len(seeds)
    pred_value_xgb += test_pred / len(seeds)

score_xgb = average_precision_score(train.target, validation_xgb)
print(score_xgb)

fold:  0
[0]	train-error:0.03273	eval-error:0.03268	train-pr_auc:-0.14573	eval-pr_auc:-0.14584
Multiple eval metrics have been passed: 'eval-pr_auc' will be used for early stopping.

Will train until eval-pr_auc hasn't improved in 10 rounds.
[50]	train-error:0.03253	eval-error:0.03255	train-pr_auc:-0.25173	eval-pr_auc:-0.25016
[100]	train-error:0.03221	eval-error:0.03216	train-pr_auc:-0.28664	eval-pr_auc:-0.28147
[150]	train-error:0.03206	eval-error:0.03210	train-pr_auc:-0.30168	eval-pr_auc:-0.29329
[200]	train-error:0.03195	eval-error:0.03210	train-pr_auc:-0.31207	eval-pr_auc:-0.29893
[250]	train-error:0.03188	eval-error:0.03206	train-pr_auc:-0.32121	eval-pr_auc:-0.30319
[300]	train-error:0.03182	eval-error:0.03204	train-pr_auc:-0.32973	eval-pr_auc:-0.30672
[350]	train-error:0.03174	eval-error:0.03203	train-pr_auc:-0.33645	eval-pr_auc:-0.30927
[400]	train-error:0.03167	eval-error:0.03201	train-pr_auc:-0.34329	eval-pr_auc:-0.31159
[450]	train-error:0.03162	eval-error:0.03198	train-pr_a

# submission

In [34]:
lgb_sub = pd.read_csv(DIR + "atmaCup7__sample_submission.csv")
lgb_sub["target"] = (pred_value+pred_value_ts+pred_value_xgb)/3
lgb_sub.to_csv("atmacup7_"+str(score)[:-10]+".csv", index = False)